## PCA and Linear Regression
I am using a combination of Bill and Jessica's code to complete my analysis properly 
I am conducting PCA to see how variable interact within the Flinn-Engdahl regions and seeing if it is possible to do linear regression with these regions. I am also going to be adding B-values from Jessica's B-value notebook to the data set.

# Data Ingestion

The following cells will be used to ingest the data needed for the PCA and linear regression and reformat and reorganize them into dataframes that will be used in my analysis

In [1]:
#Enables the use of functions from other R notebooks 
source('initialize_data.R', echo = FALSE)

In [3]:
#Installs necessary packages
install.packages('spdep')

load_libraries()

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Loading required package: sp
### Welcome to rworldmap ###
For a short introduction type : 	 vignette('rworldmap')
Checking rgeos availability: TRUE

Attaching package: ‘maps’

The fo

[1] "libraries loaded successfully"


In [4]:
#Bill
#Explicit location of main project data
path <- "~/jupyter/cs2019_Group11/GroupProducts/data" 

#create df from main project data
df <- loadFiles(path) %>%
    reqCols %>%
    parseDt('time')

#create spatial point df
dfsp <- tospdf(df)

Files in "~/jupyter/cs2019_Group11/GroupProducts/data" loaded.
Subset Completed.
Parsed column: time.
spatial df created


In [5]:
#bill
#takes about 10 minutes to run
#adds Flinn-Engdahl regions to spatial point df and converts coordinate system
tpts <-regions_data(dfsp)

OGR data source with driver: LIBKML 
Source: "/dsa/home/ewghbb/jupyter/cs2019_Group11/GroupProducts/fe.kmz", layer: "fe"
with 754 features
It has 12 fields


Warning message in readOGR(tectonicdata):
“Z-dimension discarded”

[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"


In [6]:
#jessica 
base_world <- base_world_regions()
#get base map

OGR data source with driver: LIBKML 
Source: "/dsa/home/ewghbb/jupyter/cs2019_Group11/GroupProducts/fe.kmz", layer: "fe"
with 754 features
It has 12 fields


Warning message in readOGR(regiondata):
“Z-dimension discarded”Regions defined for each Polygons
Warning message:
“Ignoring unknown parameters: fill”

In [7]:
#Jessica 
b.guten <-
function(magn,m0=min(magn)){ #
            x = subset(magn,subset=(magn>m0))
            x = x-m0
            n  = length(x)
            beta = n/sum(x)
            samplevar = beta/sum(x)
    b  = beta/log(10)
    se = sqrt(samplevar/log(10))
            return(c(b,se))
            }

In [8]:
#Bill + jessica 
#converting spatial point df back to normal df in order to plot more meaningful visualizations
tpts_df <- as.data.frame(tpts)
tpts_df$latitude = df$latitude
tpts_df$longitude = df$longitude
tpts_df$coords.x1 = NULL
tpts_df$coords.x2 = NULL

head(tpts_df)

depth,mag,time,id,updated,newname,latitude,longitude
2.000,2.6,1999-07-01 23:29:27,usp0009awp,2014-11-07 01:08:01,374,45.4500,3.790
4.613,3.1,1999-07-01 22:10:51,uw10474128,2016-07-23 18:59:57,6,46.7105,-122.778
106.800,4.2,1999-07-01 21:42:41,usp0009awk,2014-11-07 01:08:01,1,51.9950,177.972
26.100,3.9,1999-07-01 20:50:11,usp0009awh,2014-11-07 01:08:01,53,-31.7260,-72.053
136.300,3.1,1999-07-01 19:52:19,usp0009awf,2014-11-07 01:08:01,24,59.8900,-153.560
30.200,3.8,1999-07-01 19:52:12,usp0009awe,2014-11-07 01:08:01,741,36.0500,31.166


In [9]:
summary(tpts_df)

     depth             mag             time                    
 Min.   : -3.60   Min.   :2.500   Min.   :1999-01-01 01:02:39  
 1st Qu.: 10.00   1st Qu.:2.990   1st Qu.:2004-12-08 03:12:53  
 Median : 22.00   Median :4.000   Median :2008-12-20 11:33:42  
 Mean   : 54.59   Mean   :3.794   Mean   :2009-08-02 09:43:03  
 3rd Qu.: 50.00   3rd Qu.:4.500   3rd Qu.:2014-10-24 01:16:41  
 Max.   :735.80   Max.   :9.100   Max.   :2018-12-31 23:54:07  
      id               updated                      newname         
 Length:511837      Min.   :2013-02-06 02:29:15   Length:511837     
 Class :character   1st Qu.:2014-11-07 01:25:49   Class :character  
 Mode  :character   Median :2014-11-07 01:48:07   Mode  :character  
                    Mean   :2016-02-23 13:07:26                     
                    3rd Qu.:2017-07-29 03:16:10                     
                    Max.   :2019-06-28 18:47:47                     
    latitude         longitude      
 Min.   :-84.422   Min.   :-180.

In [10]:
#Jessica
 tectonicdata = "~/jupyter/cs2019_Group11/GroupProducts/fe.kmz"
    tectonicFeatures <- readOGR(tectonicdata)

    #also transform data into same regions as the earthquake data for later comparisons
    transTectonicFeatures <- spTransform(tectonicFeatures,  CRS("+init=epsg:4087"))


tectonicFeatures@data$ID <- seq.int(nrow(transTectonicFeatures))#this adds in an id which is the same 
#as the newname in the tpts to match regions with earthquakes

summary(tectonicFeatures)


OGR data source with driver: LIBKML 
Source: "/dsa/home/ewghbb/jupyter/cs2019_Group11/GroupProducts/fe.kmz", layer: "fe"
with 754 features
It has 12 fields


Warning message in readOGR(tectonicdata):
“Z-dimension discarded”

Object of class SpatialPolygonsDataFrame
Coordinates:
   min max
x -180 180
y  -90  90
Is projected: FALSE 
proj4string :
[+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0]
Data attributes:
     Name           description         timestamp            begin          
 Length:754         Length:754         Length:754         Length:754        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                                            
                                                                            
     end            altitudeMode         tessellate    extrude    visibility
 Length:754         Length:754         Min.   :-1   Min.   :0   Min.   :-1  
 Class :character   Class

In [11]:
bvalue_df = read.csv("data.csv",header=TRUE,sep=",")

head(bvalue_df)
# here the data created from the bvalue_model notebook will be loaded in order to save computational time

X,newname,depth,mag,time,id,updated,latitude,longitude,bvalue,error,time_lag
1,10,33.0,5.1,1999-02-14 17:17:27,usp00092tz,2014-11-07 01:07:01,21.581,-106.681,0.6204207,0.66570086,NA
2,10,33.0,4.5,1999-03-16 06:14:07,usp00094kr,2014-11-07 01:07:14,22.002,-107.375,0.6204207,0.66570086,1999-02-14 17:17:27
3,10,10.0,4.5,1999-01-13 07:38:50,usp0009114,2014-11-07 01:06:48,22.690,-107.988,0.6204207,0.66570086,1999-03-16 06:14:07
4,10,10.0,5.3,1999-03-12 07:07:40,usp00094by,2016-11-09 22:04:11,22.219,-107.368,0.6204207,0.66570086,1999-01-13 07:38:50
5,10,33.0,5.2,1999-01-10 20:36:06,usp00090x3,2016-11-09 21:38:36,22.939,-108.020,0.6204207,0.66570086,1999-03-12 07:07:40
6,100,129.1,4.3,1999-04-15 14:05:21,usp00096bk,2014-11-07 01:07:28,-19.295,-69.268,0.3347563,0.04778564,NA


In [12]:
#Elysa
#Final Cleaning of data for PCA
bvalue_df = parseDt(bvalue_df,'time')
bvalue_df$time = NULL
head(bvalue_df)

ERROR: Error in format.default(df[[col]], "%Y"): invalid 'trim' argument


In [18]:
str(bvalue_df)

'data.frame':	506200 obs. of  12 variables:
 $ X        : int  1 2 3 4 5 6 7 8 9 10 ...
 $ newname  : int  10 10 10 10 10 100 100 100 100 100 ...
 $ depth    : num  33 33 10 10 33 ...
 $ mag      : num  5.1 4.5 4.5 5.3 5.2 4.3 3.9 4.1 3.5 3.9 ...
 $ time     : chr  "1999-02-14 17:17:27" "1999-03-16 06:14:07" "1999-01-13 07:38:50" "1999-03-12 07:07:40" ...
 $ id       : chr  "usp00092tz" "usp00094kr" "usp0009114" "usp00094by" ...
 $ updated  : chr  "2014-11-07 01:07:01" "2014-11-07 01:07:14" "2014-11-07 01:06:48" "2016-11-09 22:04:11" ...
 $ latitude : num  21.6 22 22.7 22.2 22.9 ...
 $ longitude: num  -107 -107 -108 -107 -108 ...
 $ bvalue   : num  0.62 0.62 0.62 0.62 0.62 ...
 $ error    : num  0.666 0.666 0.666 0.666 0.666 ...
 $ time_lag : chr  NA "1999-02-14 17:17:27" "1999-03-16 06:14:07" "1999-01-13 07:38:50" ...


In [13]:
#Bill + Elysa 
#finding top ten Flinn-Engdahl regions by max magnitude occuring in that region

#grouping and summarising by max mag
tpts_df_mm = bvalue_df %>% group_by(newname) %>% summarise(max(mag))

#changing column name to max_mag for easier use
colnames(tpts_df_mm) <- c('newname', 'max_mag')

#ordering by descending max_mag
tpts_df_mm = arrange(tpts_df_mm, desc(max_mag))

#displaying top then regions and their max_mags
top_ten_mag_regions = head(tpts_df_mm, 10)
top_ten_mag_regions

newname,max_mag
197,9.1
570,9.1
54,8.8
593,8.6
65,8.4
191,8.4
171,8.3
553,8.3
669,8.3
60,8.2


In [14]:
#Elysa 
#creates the bottom ten regions
tpts_df_mm2 = arrange(tpts_df_mm,max_mag)
bottom_ten_mag_regions = head(tpts_df_mm2, 10)
bottom_ten_mag_regions

newname,max_mag
518,2.60
597,2.60
430,2.70
478,2.80
400,2.90
622,2.90
425,2.95
740,2.98
384,3.00
432,3.00


In [15]:
#Elysa
#Creating a new dataframe with only the top ten regions 
keep <- c("197","570","54","593","191","65","171","553","669","60")
tpts_df = bvalue_df[bvalue_df$newname %in% keep, ]
head(tpts_df)
dim(tpts_df)

,X,newname,depth,mag,time,id,updated,latitude,longitude,bvalue,error,time_lag
3050,3050,171,33.0,4.7,1999-05-19 06:31:33,usp00098am,2014-11-07 01:07:41,46.786,151.835,0.2822652,0.01414427,NA
3051,3051,171,72.6,4.7,1999-07-22 08:57:54,usp0009byj,2014-11-07 01:08:07,43.883,147.124,0.2822652,0.01414427,1999-05-19 06:31:33
3052,3052,171,45.3,4.6,1999-08-29 21:12:52,usp0009dt8,2014-11-07 01:08:21,46.655,152.748,0.2822652,0.01414427,1999-07-22 08:57:54
3053,3053,171,48.0,4.8,1999-05-28 19:50:28,usp00098xn,2014-11-07 01:07:43,45.340,151.272,0.2822652,0.01414427,1999-08-29 21:12:52
3054,3054,171,33.0,5.3,1999-02-14 11:22:37,usp00092t8,2016-11-09 21:53:03,44.506,149.710,0.2822652,0.01414427,1999-05-28 19:50:28
3055,3055,171,33.0,4.5,1999-06-20 03:35:03,usp0009a6t,2014-11-07 01:07:55,46.304,151.618,0.2822652,0.01414427,1999-02-14 11:22:37


[1] 35942    12

In [16]:
#Elysa
#Finds the unique values of the highest magnitude regions in the dataframe 
regions = unique(top_ten_mag_regions$newname)
regions

[1] 197 570  54 593  65 191 171 553 669  60

In [17]:
#Final check of the data and then creating more time variables
bvalue_df$bvalue <- as.numeric(bvalue_df$bvalue)
bvalue_df$time_Time <- as.numeric(bvalue_df$time_Time)
str(bvalue_df)

ERROR: Error in `$<-.data.frame`(`*tmp*`, time_Time, value = numeric(0)): replacement has 0 rows, data has 506200


In [21]:
#Elysa
#Testing to see if I can create dataframes basing on unique regions using a for loop to save time 
#After the dataframes are seperated by region then I can create a PCA for each one 
for (i in c(1,2,3,4,5,6,7,8,9,10)) {
  assign(paste0("df_", i), subset(bvalue_df, newname == regions[i],select=c(depth,mag,time_Year,time_Time)) )
}

dflist = list(df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_9,df_10)

for (i in 1:length(dflist)) {
  assign(paste0("PCA_", i),prcomp(dflist[[i]], center = TRUE,scale. = TRUE ))
}

In [22]:
#Elysa
#Testing to see if the PCA was created for all ten values 
summary(PCA_10)

Importance of components:
                          PC1    PC2    PC3   PC4
Standard deviation     1.1316 1.0061 0.9855 0.858
Proportion of Variance 0.3201 0.2530 0.2428 0.184
Cumulative Proportion  0.3201 0.5732 0.8160 1.000

In [25]:
#Elysa
#Created a base function to create PCA based on region
#Region can be changed to continent or whatever variable you'd like to use 
PCAFunc = function(df){
    regions = unique(df$newname)
regions
for (i in c(1,2,3,4,5,6,7,8,9,10)) {
  assign(paste0("df_", i), subset(df, newname == regions[i],select=c(newname,depth,mag,time_Year,time_Time)) )
}

dflist = list(df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_9,df_10)

for (i in 1:length(dflist)) {
  assign(paste0("PCA_", i),prcomp(dflist[[i]], center = TRUE ))
}
    }

In [26]:
#testing to see if my PCA function worked 
PCAFunc(bvalue_df)
summary(PCA_10)

ERROR: Error in colMeans(x, na.rm = TRUE): 'x' must be numeric


In [ ]:
#importing libraries to visualize the PCA
library(devtools)
options(unzip = "internal")
devtools::install_github("vqv/ggbiplot")

In [ ]:
#importing the library for plotting PCA 
library(ggbiplot)

#Created a list of PCAs for each top ten highest magnitude region to iterate through
pcalist = list(PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10)

for (i in 1:length(pcalist)) {
  print(ggbiplot(pcalist[[i]],alpha = 0.05))
}

#PC1 is Depth and PC2 is Mag

With the addition of the time variable we can see how the explanation of variances changes from region to region. 
B-values could not be included in the PCA due to each region having the same b-value making the variance equal to 0.
Now I will look at the PCA for the bottom most regions.

In [ ]:
#Elysa
#Creating a new dataframe with only the bottom ten regions 
keep <- c("518 ","597 ","430","478","400","622","425","740","384","387")
tpts_df2 = bvalue_df[bvalue_df$newname %in% keep, ]
tpts_df2[is.na(tpts_df2)] <- 0
tpts_df2

In [ ]:
#Elysa
#Finds the unique values of the lowest magnitude regions in the dataframe 
regions = unique(bottom_ten_mag_regions$newname)
regions

In [ ]:
#Checking to make sure time is an integer value so it works for PCA
tpts_df2$time_Time <- as.numeric(tpts_df2$time_Time)
str(tpts_df2)

In [ ]:
#testing to see if my PCA function worked 
PCAFunc(tpts_df2)
summary(PCA_10)

In [ ]:
head(df_1)

Not sure why it will not work for the lowest magnitude regions, I will need to ask my group for assistance with this. 

In [ ]:
#It worked once for the bottom most regions but then it never worked again
#importing the library for plotting PCA 
library(ggbiplot)

#Created a list of PCAs for each bottom most region to iterate through
pcalist = list(PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10)

for (i in 1:length(pcalist)) {
  print(ggbiplot(pcalist[[i]],alpha = 0.05))
}

#PC1 is Depth and PC2 is Mag

# Multiple Linear Regression
Now I will do multiple linear regression for the top magnitude and bottom magnitude regions. I will only perform the regression on regions that have most of their variables have a correlation rate over 10%. 

In [ ]:
#Elysa
#Finds the unique values of the highest magnitude regions in the dataframe 
regions = unique(top_ten_mag_regions$newname)
regions

In [ ]:
#Now I will make 10 dataframes with only numeric data for multiple linear regression for each highest magnitude region
for (i in c(1,2,3,4,5,6,7,8,9,10)) {
  assign(paste0("lindf_", i), subset(bvalue_df, newname == regions[i],select=c(depth,mag,latitude,longitude,time_Year,time_Time,bvalue)) )
}
head(lindf_1)

In [ ]:
#Creating a list to iterate through and do correlation for all variables
linlist = list(lindf_1,lindf_2,lindf_3,lindf_4,lindf_5,lindf_6,lindf_7,lindf_8,lindf_9,lindf_10)

for (i in 1:length(linlist)) {
  print("Corr Matrix for Region:",i)
  print(cor(linlist[[i]]))
}

#After looking at correlation it appears that there is a lot of variation between regions on correlation 
#values for our predictor values magnitude. Correlation is strongest for corr matrix three, or Region 54

In [ ]:
library(caret)
#Multiple Linear Regression for Region 54
#Based on the correlation matrix I will keep all variable in the analysis
formula1 <- mag~.
m1 <- train(formula1, lindf_3, method = "lm")
summary(m1)

Similar to last time, even with the time and b-value added in the best I can do is an R^2 of 0.3272

In [ ]:
#Checking to see what predicted values look like next to actual mag values 
coef.icept <- coef(m1$finalModel)[1] # define the intercept
coef.slope <- coef(m1$finalModel)[2] # define the slope
class_coefs <- coef(m1$finalModel)[2:8] # define adjusted slopes per class
lindf_3$pred<-predict(m1, newdata=lindf_3) # find the predicted values of the dataset mag
head(lindf_3)

In [ ]:
#Trying to see if a logisitic regression would work better using region 54
lindf_log = glm(mag ~ ., data=lindf_3, family=poisson)
summary(lindf_log)

In [ ]:
#Checking the accuracy of the model
probs = predict(lindf_log, type = "response", newdata=lindf_3)
preds <- ifelse(probs > 0.5,1,0)
misClassificError <- mean (preds != probs)
print(paste('Accuracy',1-misClassificError))

Both linear and logistic regression do not seem to work well with the data and the b-values and time do not add to the accuracy.

In [ ]:
#Findigng the bottom magnitude regions
regions = unique(bottom_ten_mag_regions$newname)
regions

In [ ]:
#Now I will make 10 dataframes with only numeric data for multiple linear regression for each bottom most region
for (i in c(1,2,3,4,5,6,7,8,9,10)) {
  assign(paste0("lindf_", i), subset(bvalue_df, newname == regions[i],select=c(depth,mag,latitude,longitude,time_Year,time_Time,bvalue)) )
}
head(lindf_1)

In [ ]:
#Creating a list to iterate through and do correlation for all variables
linlist = list(lindf_1,lindf_2,lindf_3,lindf_4,lindf_5,lindf_6,lindf_7,lindf_8,lindf_9,lindf_10)

for (i in 1:length(linlist)) {
  print("Corr Matrix for Region:",i)
  print(cor(linlist[[i]]))
}

#After looking at correlation it appears that there is a lot of variation between regions on correlation 
#values for our predictor values magnitude. Correlation is strongest for corr matrix 10, or Region 387 and matrix3 or region 430

In [ ]:
#Multiple Linear Regression for Region 387
#Based on the correlation matrix I will keep all variable in the analysis
formula3 <- mag~.
m3 <- train(formula3, lindf_3, method = "lm")
summary(m3)

In [ ]:
#Trying to see if a logisitic regression would work better using region 387
lindf_log = glm(mag ~ ., data=lindf_3, family=poisson)
summary(lindf_log)

In [ ]:
#Checking the accuracy of the model
probs = predict(lindf_log, type = "response", newdata=lindf_3)
preds <- ifelse(probs > 0.5,1,0)
misClassificError <- mean (preds != probs)
print(paste('Accuracy',1-misClassificError))

In [ ]:
#Multiple Linear Regression for Region 430
#Based on the correlation matrix I will keep all variable in the analysis
formula10 <- mag~.
m10 <- train(formula3, lindf_10, method = "lm")
summary(m10)

In [ ]:
#Trying to see if a logisitic regression would work better using region 430
lindf_log = glm(mag ~ ., data=lindf_10, family=poisson)
summary(lindf_log)

In [ ]:
#Checking the accuracy of the model
probs = predict(lindf_log, type = "response", newdata=lindf_10)
preds <- ifelse(probs > 0.5,1,0)
misClassificError <- mean (preds != probs)
print(paste('Accuracy',1-misClassificError))

I think due to the lack of data for the bottom most regions makes it hard to use these regions for PCA. Both regions that showed high correlation did not work for multipe linear regression or logistic regrssion. Using the bottom most regions may not work for this type of analysis.